Условия

Если задача не отображается, то вы либо не авторизованы, либо не нажали кнопку кнопку «принять участие» на этой странице.

Для данной задачи вам предстоит построить рекомендательную систему баннеров на основе логов просмотров и лайков.

Логи представлены четырьмя колонками:

user_id (идентификатор пользователя),
item_id (идентификатор баннера),
like (флаг понравился ли пользователю баннер),
timestamp (unix время в секундах совершения действия).
Кроме того, для пользователей и баннеров имеются признаки размерностью 32.

Вам необходимо предсказать 20 баннеров для пользователей. Качество решения будет оцениваться как доля "лайкнутых" пользователей баннеров из предложенного вами списка (top-20 accuracy).

Описание файлов
test.csv — тестовый файл, содержащий идентификаторы пользователи, для которых необходимо сделать предсказания

train.csv — обучающий датасет

item-features.csv — признаки для баннеров

​​user-features — признаки для пользователей

sample-submission.csv — пример решения (сабмита).

качество решения оценивается по метрике Top-K Accuracy, где k = 20. Код:

In [1]:
def calc_score(test_choices, pred_choices, tk):
    s = 0
    for gt, p in zip(test_choices, pred_choices):
        s += int(gt in p)
    score = s / len(test_choices)
    return score

На экзамене за данную задачу вы можете получить до 100 баллов. Расчёт баллов производится по формуле (отличается от задачи 1 коэффициентами):

In [2]:
y = 0
result = 100 if y > 0.54 else (40 + (y - 0.5) / 0.04 * 60 if y > 0.5 else 0)

# Libs

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_validate

from sklearn.model_selection import train_test_split

import seaborn

где y — это ваш результат по метрике Top-K Accuracy. Количество баллов является округленным целым числом.

Желаем удачи!

In [4]:
!ls

docs		       recommend.ipynb	      train.csv
item-features.csv      sample-submission.csv  user-features.csv
recommend-Copy1.ipynb  test.csv


# Data description

## Matrix items-features

In [5]:
item_features = (pd.read_csv("item-features.csv")).sort_values(by = ['item_id'])
item_features

,item_id,0,1,2,3,4,5,6,7,8,...,22,23,24,25,26,27,28,29,30,31
388,0,0.001433,-0.003243,-0.003030,0.004299,-0.001026,0.001412,0.001671,0.001373,-0.006249,...,-0.008651,-0.001440,0.002312,-0.002225,-0.004108,-0.004108,0.000871,-0.002408,-0.002408,0.000613
169,1,0.002482,-0.005617,-0.005248,0.007446,-0.001777,0.002446,0.002895,0.002378,-0.010824,...,-0.014983,-0.002493,0.004004,-0.003855,-0.007115,-0.007115,0.001508,-0.004171,-0.004171,0.001062
239,2,0.001871,-0.004236,-0.003958,0.005615,-0.001340,0.001845,0.002183,0.001793,-0.008162,...,-0.011299,-0.001880,0.003020,-0.002907,-0.005365,-0.005365,0.001137,-0.003145,-0.003145,0.000801
425,3,0.001390,-0.003146,-0.002940,0.004171,-0.000996,0.001370,0.001622,0.001332,-0.006063,...,-0.008392,-0.001397,0.002243,-0.002159,-0.003985,-0.003985,0.000845,-0.002336,-0.002336,0.000595
260,4,0.001738,-0.003933,-0.003675,0.005213,-0.001244,0.001713,0.002027,0.001665,-0.007579,...,-0.010490,-0.001746,0.002804,-0.002699,-0.004982,-0.004982,0.001056,-0.002920,-0.002920,0.000743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,439,0.000348,-0.000787,-0.000735,0.001043,-0.000249,0.000343,0.000405,0.000333,-0.001516,...,-0.002098,-0.000349,0.000561,-0.000540,-0.000996,-0.000996,0.000211,-0.000584,-0.000584,0.000149
346,440,0.000777,-0.001759,-0.001643,0.002332,-0.000557,0.000766,0.000906,0.000745,-0.003389,...,-0.004691,-0.000781,0.001254,-0.001207,-0.002228,-0.002228,0.000472,-0.001306,-0.001306,0.000332
37,441,-0.121692,0.181685,0.204412,-0.150535,0.235095,-0.389399,-0.326175,0.287157,-0.107230,...,-0.035643,-0.015009,-0.286772,0.002378,-0.065991,-0.065991,-0.112601,0.159891,0.159891,-0.370836
178,442,0.000491,-0.001112,-0.001039,0.001475,-0.000352,0.000484,0.000573,0.000471,-0.002144,...,-0.002967,-0.000494,0.000793,-0.000763,-0.001409,-0.001409,0.000299,-0.000826,-0.000826,0.000210


In [6]:
item_features["item_id"].astype(object).describe()[['count', 'unique']]

count     444
unique    444
Name: item_id, dtype: int64

In [7]:
print(f"444 items and 32 features")

444 items and 32 features


In [8]:
matrix_item_features = item_features[item_features.columns[1:]].values
matrix_item_features.shape

(444, 32)

## Matrix user-features

In [9]:
user_features = (pd.read_csv("user-features.csv")).sort_values(by=['user_id'])
user_features

,user_id,0,1,2,3,4,5,6,7,8,...,22,23,24,25,26,27,28,29,30,31
0,0,0.000695,-0.001573,-0.001470,0.002085,-0.000498,0.000685,0.000811,0.000666,-0.003031,...,-0.004196,-0.000698,0.001121,-0.001079,-0.001993,-0.001993,0.000422,-0.001168,-0.001168,0.000297
1,1,0.001204,-0.002725,-0.002546,0.003612,-0.000862,0.001187,0.001404,0.001154,-0.005251,...,-0.007268,-0.001209,0.001942,-0.001870,-0.003451,-0.003451,0.000732,-0.002023,-0.002023,0.000515
2,2,0.000491,-0.001112,-0.001039,0.001475,-0.000352,0.000484,0.000573,0.000471,-0.002144,...,-0.002967,-0.000494,0.000793,-0.000763,-0.001409,-0.001409,0.000299,-0.000826,-0.000826,0.000210
3,3,0.000777,-0.001759,-0.001643,0.002332,-0.000557,0.000766,0.000906,0.000745,-0.003389,...,-0.004691,-0.000781,0.001254,-0.001207,-0.002228,-0.002228,0.000472,-0.001306,-0.001306,0.000332
4,4,0.000695,-0.001573,-0.001470,0.002085,-0.000498,0.000685,0.000811,0.000666,-0.003031,...,-0.004196,-0.000698,0.001121,-0.001079,-0.001993,-0.001993,0.000422,-0.001168,-0.001168,0.000297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,492,0.000983,-0.002225,-0.002079,0.002949,-0.000704,0.000969,0.001147,0.000942,-0.004287,...,-0.005934,-0.000988,0.001586,-0.001527,-0.002818,-0.002818,0.000597,-0.001652,-0.001652,0.000421
493,493,0.001300,-0.002943,-0.002750,0.003901,-0.000931,0.001282,0.001517,0.001246,-0.005671,...,-0.007850,-0.001306,0.002098,-0.002020,-0.003728,-0.003728,0.000790,-0.002185,-0.002185,0.000556
494,494,0.000491,-0.001112,-0.001039,0.001475,-0.000352,0.000484,0.000573,0.000471,-0.002144,...,-0.002967,-0.000494,0.000793,-0.000763,-0.001409,-0.001409,0.000299,-0.000826,-0.000826,0.000210
495,495,0.000983,-0.002225,-0.002079,0.002949,-0.000704,0.000969,0.001147,0.000942,-0.004287,...,-0.005934,-0.000988,0.001586,-0.001527,-0.002818,-0.002818,0.000597,-0.001652,-0.001652,0.000421


In [10]:
user_features["user_id"].astype(object).describe()[['count', 'unique']]

count     497
unique    497
Name: user_id, dtype: int64

In [11]:
print(f"497 items and 32 features")

497 items and 32 features


In [12]:
matrix_user_features = user_features[user_features.columns[1:]].values
matrix_user_features.shape

(497, 32)

## logs

In [13]:
logs = (pd.read_csv("train.csv")).sort_values(by = ['timestamp'])
logs["timestamp"]= pd.to_datetime(logs["timestamp"]*10**9)
logs

,user_id,item_id,like,timestamp
0,140,342,0,2017-03-31 05:03:42
1,378,172,1,2017-03-31 05:03:48
2,150,182,0,2017-03-31 05:04:10
3,455,17,0,2017-03-31 05:05:04
4,350,409,0,2017-03-31 05:05:35
...,...,...,...,...
8669,161,312,0,2017-04-03 10:31:59
8670,406,208,0,2017-04-03 10:32:23
8671,196,43,0,2017-04-03 10:32:56
8672,84,100,0,2017-04-03 10:32:59


In [14]:
logs["user_id"].astype(object).describe()[['count', 'unique']]

count     8674
unique     497
Name: user_id, dtype: int64

In [15]:
logs["item_id"].astype(object).describe()[['count', 'unique']]

count     8674
unique     444
Name: item_id, dtype: int64

In [16]:
logs["like"].value_counts()

0    7148
1    1526
Name: like, dtype: int64

In [17]:
logs["timestamp"].describe()[['count','first', 'last']]

count                   8674
first    2017-03-31 05:03:42
last     2017-04-03 10:33:15
Name: timestamp, dtype: object

In [18]:
logs['user_id item_id like'.split()]

,user_id,item_id,like
0,140,342,0
1,378,172,1
2,150,182,0
3,455,17,0
4,350,409,0
...,...,...,...
8669,161,312,0
8670,406,208,0
8671,196,43,0
8672,84,100,0


## Test table for recommendation

In [19]:
test = pd.read_csv("test.csv").sort_values(by = ['timestamp'])
test["timestamp"] = pd.to_datetime(test["timestamp"]*10**9)
test

,user_id,timestamp
0,166,2017-03-31 07:13:51
1,26,2017-03-31 10:49:31
2,41,2017-03-31 11:02:27
3,286,2017-03-31 14:40:55
4,108,2017-03-31 16:13:56
...,...,...
492,190,2017-04-03 10:20:14
493,181,2017-04-03 10:20:29
494,448,2017-04-03 10:22:08
495,124,2017-04-03 10:26:37


In [20]:
test["user_id"].astype(object).describe()[['count', 'unique']]

count     497
unique    497
Name: user_id, dtype: int64

In [21]:
test["timestamp"].describe()[['count','first', 'last']]

count                    497
first    2017-03-31 07:13:51
last     2017-04-03 10:30:21
Name: timestamp, dtype: object

In [22]:
# get k indexes with to k values
def get_predictions(A :np.array, top_k=20):
    result = np.zeros(shape = (len(A), top_k), dtype = int)
    for row in range(len(A)):
        values = [(x, i) for i, x in enumerate(A[row])]
        max_values = sorted(values, key = lambda x: -x[0])[:top_k]
#         print(max_values)
        indexes = [x[1] for x in max_values]
        result[row] = indexes
    return result

A = np.array([[6,2,3,4,5, 2000, 20, 2, 2000, 2000], [500,9,8,7,11,100, 2000, 2, 2000, 200]])
get_predictions(A, 3)

array([[5, 8, 9],
       [6, 8, 0]])

# answers

In [23]:
np.linspace(0.2, 1.5, 14)

array([0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3, 1.4,
       1.5])

In [34]:
class MyEstimator():
    def __init__(self, matrix_user_features, matrix_item_features, w0=0, w1=0, w2=0, w3=0):
        self.w0 = w0
        self.w1 = w1
        self.w2 = w2
        self.w3 = w3
        self.matrix_user_features = matrix_user_features
        self.matrix_item_features = matrix_item_features

    def fit(X, y):
        self.logs = X
        self.y = y
        return self

    def predict(Z):
        UF = self.matrix_user_features
        IF = self.matrix_item_features
        # filter logs by time
        time_str = Z.iat[row, 3]
        logs_by_current_time = logs[logs['timestamp'] < pd.Timestamp(
            time_str)].sorted_values(by=['timestamp'])
        for row in range(len(logs_by_current_time)):
            user_id = logs_by_current_time.iat[row, 0]
            item_id = logs_by_current_time.iat[row, 1]
            like_value = logs_by_current_time.iat[row, 2]
            time = logs_by_current_time.iat[row, 3]

#             predictions = get_predictions([UF[user_id].dot(IF.T)])

#             if item_id in predictions:
#                 score += 1
            # new UF by user_id click on iten_id
            if like_value == 1:
                UF[user_id] = self.w0*UF[user_id] + self.w1*IF[item_id]
            if like_value == 0:
                UF[user_id] = self.w2*UF[user_id] + self.w3*IF[item_id]
                
            predictions = get_predictions([UF[user_id].dot(IF.T)])
            
            if item_id in predictions and  like_value ==1 :
                return 1
            else:
                return 0
            

In [35]:
est = MyEstimator(matrix_user_features, matrix_item_features)

In [36]:
from sklearn.model_selection import RandomizedSearchCV
params = {
    'w0': np.linspace(0, 1.5, 16),
    'w1': np.linspace(0, 1.5, 16),
    'w2': np.linspace(0, 1.5, 16),
    'w3': np.linspace(0, 1.5, 16),
}

In [37]:
clf = RandomizedSearchCV(est, params, random_state=0, n_jobs=4,scoring = 'accuracy')
search = clf.fit(logs, logs['like'])
search.best_params_

TypeError: Cannot clone object '<__main__.MyEstimator object at 0x7fbd027a9350>' (type <class '__main__.MyEstimator'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [ ]:
from tqdm.notebook import tqdm
scores = []
for w0 in tqdm(np.linspace(0, 1.5, 16)):
    for w1 in np.linspace(0.2, 1.5, 14):
        for w2 in np.linspace(0.2, 1, 9):
            for w3 in np.linspace(0.2, 1.5, 14):
                UF = matrix_user_features
                IF = matrix_item_features
                score = 0

                for row in range(len(logs)):
                    user_id = logs.iat[row, 0]
                    item_id = logs.iat[row, 1]
                    like_value = logs.iat[row, 2]
                    time = logs.iat[row, 3]
                    
                    predictions = get_predictions([UF[user_id].dot(IF.T)])
                    
                    if item_id in predictions:
                        score += 1
                    # new UF by user_id click on iten_id
                    if like_value == 1:
                        UF[user_id] = w0*UF[user_id] + w1*IF[item_id]
                    if like_value == 0:
                        UF[user_id] = w2*UF[user_id] + w3*IF[item_id]

                
#                 scores.append((score,  w1, w2))
#                 print((score, w1, w2))
                print( f'{w0:.2f}, {w1:.2f}, {w2:.2f}, {w3:.2f}, {score}')
                scores.append((score, w0 , w1, w2, w3))

0.00, 0.20, 0.20, 0.20, 742
0.00, 0.20, 0.20, 0.30, 741
0.00, 0.20, 0.20, 0.40, 741
0.00, 0.20, 0.20, 0.50, 741
0.00, 0.20, 0.20, 0.60, 741
0.00, 0.20, 0.20, 0.70, 741
0.00, 0.20, 0.20, 0.80, 741
0.00, 0.20, 0.20, 0.90, 741
0.00, 0.20, 0.20, 1.00, 741
0.00, 0.20, 0.20, 1.10, 741
0.00, 0.20, 0.20, 1.20, 741
0.00, 0.20, 0.20, 1.30, 741
0.00, 0.20, 0.20, 1.40, 741
0.00, 0.20, 0.20, 1.50, 741
0.00, 0.20, 0.30, 0.20, 743
0.00, 0.20, 0.30, 0.30, 741
0.00, 0.20, 0.30, 0.40, 741
0.00, 0.20, 0.30, 0.50, 741
0.00, 0.20, 0.30, 0.60, 741
0.00, 0.20, 0.30, 0.70, 741
0.00, 0.20, 0.30, 0.80, 741
0.00, 0.20, 0.30, 0.90, 741
0.00, 0.20, 0.30, 1.00, 741
0.00, 0.20, 0.30, 1.10, 741
0.00, 0.20, 0.30, 1.20, 741
0.00, 0.20, 0.30, 1.30, 741
0.00, 0.20, 0.30, 1.40, 741
0.00, 0.20, 0.30, 1.50, 741
0.00, 0.20, 0.40, 0.20, 743
0.00, 0.20, 0.40, 0.30, 741
0.00, 0.20, 0.40, 0.40, 741
0.00, 0.20, 0.40, 0.50, 741
0.00, 0.20, 0.40, 0.60, 741
0.00, 0.20, 0.40, 0.70, 741
0.00, 0.20, 0.40, 0.80, 741
0.00, 0.20, 0.40, 0.

0.00, 0.40, 0.40, 1.50, 741
0.00, 0.40, 0.50, 0.20, 744
0.00, 0.40, 0.50, 0.30, 742
0.00, 0.40, 0.50, 0.40, 743
0.00, 0.40, 0.50, 0.50, 743
0.00, 0.40, 0.50, 0.60, 743
0.00, 0.40, 0.50, 0.70, 743
0.00, 0.40, 0.50, 0.80, 743
0.00, 0.40, 0.50, 0.90, 743
0.00, 0.40, 0.50, 1.00, 743
0.00, 0.40, 0.50, 1.10, 743
0.00, 0.40, 0.50, 1.20, 743
0.00, 0.40, 0.50, 1.30, 743
0.00, 0.40, 0.50, 1.40, 743
0.00, 0.40, 0.50, 1.50, 743
0.00, 0.40, 0.60, 0.20, 744
0.00, 0.40, 0.60, 0.30, 745
0.00, 0.40, 0.60, 0.40, 745
0.00, 0.40, 0.60, 0.50, 745
0.00, 0.40, 0.60, 0.60, 745
0.00, 0.40, 0.60, 0.70, 745
0.00, 0.40, 0.60, 0.80, 745
0.00, 0.40, 0.60, 0.90, 745
0.00, 0.40, 0.60, 1.00, 745
0.00, 0.40, 0.60, 1.10, 745
0.00, 0.40, 0.60, 1.20, 745
0.00, 0.40, 0.60, 1.30, 745
0.00, 0.40, 0.60, 1.40, 745
0.00, 0.40, 0.60, 1.50, 745
0.00, 0.40, 0.70, 0.20, 746
0.00, 0.40, 0.70, 0.30, 746
0.00, 0.40, 0.70, 0.40, 746
0.00, 0.40, 0.70, 0.50, 746
0.00, 0.40, 0.70, 0.60, 746
0.00, 0.40, 0.70, 0.70, 746
0.00, 0.40, 0.70, 0.

0.00, 0.60, 0.70, 1.40, 747
0.00, 0.60, 0.70, 1.50, 747
0.00, 0.60, 0.80, 0.20, 746
0.00, 0.60, 0.80, 0.30, 747
0.00, 0.60, 0.80, 0.40, 747
0.00, 0.60, 0.80, 0.50, 747
0.00, 0.60, 0.80, 0.60, 748
0.00, 0.60, 0.80, 0.70, 748
0.00, 0.60, 0.80, 0.80, 747
0.00, 0.60, 0.80, 0.90, 747
0.00, 0.60, 0.80, 1.00, 747
0.00, 0.60, 0.80, 1.10, 747
0.00, 0.60, 0.80, 1.20, 747
0.00, 0.60, 0.80, 1.30, 747
0.00, 0.60, 0.80, 1.40, 747
0.00, 0.60, 0.80, 1.50, 747
0.00, 0.60, 0.90, 0.20, 746
0.00, 0.60, 0.90, 0.30, 745
0.00, 0.60, 0.90, 0.40, 745
0.00, 0.60, 0.90, 0.50, 746
0.00, 0.60, 0.90, 0.60, 746
0.00, 0.60, 0.90, 0.70, 746
0.00, 0.60, 0.90, 0.80, 746
0.00, 0.60, 0.90, 0.90, 746
0.00, 0.60, 0.90, 1.00, 746
0.00, 0.60, 0.90, 1.10, 746
0.00, 0.60, 0.90, 1.20, 746
0.00, 0.60, 0.90, 1.30, 746
0.00, 0.60, 0.90, 1.40, 746
0.00, 0.60, 0.90, 1.50, 746
0.00, 0.60, 1.00, 0.20, 745
0.00, 0.60, 1.00, 0.30, 744
0.00, 0.60, 1.00, 0.40, 744
0.00, 0.60, 1.00, 0.50, 746
0.00, 0.60, 1.00, 0.60, 746
0.00, 0.60, 1.00, 0.

0.00, 0.80, 1.00, 1.30, 746
0.00, 0.80, 1.00, 1.40, 746
0.00, 0.80, 1.00, 1.50, 746
0.00, 0.90, 0.20, 0.20, 742
0.00, 0.90, 0.20, 0.30, 741
0.00, 0.90, 0.20, 0.40, 741
0.00, 0.90, 0.20, 0.50, 741
0.00, 0.90, 0.20, 0.60, 741
0.00, 0.90, 0.20, 0.70, 741
0.00, 0.90, 0.20, 0.80, 741
0.00, 0.90, 0.20, 0.90, 741
0.00, 0.90, 0.20, 1.00, 741
0.00, 0.90, 0.20, 1.10, 741
0.00, 0.90, 0.20, 1.20, 741
0.00, 0.90, 0.20, 1.30, 741
0.00, 0.90, 0.20, 1.40, 741
0.00, 0.90, 0.20, 1.50, 741
0.00, 0.90, 0.30, 0.20, 743
0.00, 0.90, 0.30, 0.30, 741
0.00, 0.90, 0.30, 0.40, 741
0.00, 0.90, 0.30, 0.50, 741
0.00, 0.90, 0.30, 0.60, 741
0.00, 0.90, 0.30, 0.70, 741
0.00, 0.90, 0.30, 0.80, 741
0.00, 0.90, 0.30, 0.90, 741
0.00, 0.90, 0.30, 1.00, 741
0.00, 0.90, 0.30, 1.10, 741
0.00, 0.90, 0.30, 1.20, 741
0.00, 0.90, 0.30, 1.30, 741
0.00, 0.90, 0.30, 1.40, 741
0.00, 0.90, 0.30, 1.50, 741
0.00, 0.90, 0.40, 0.20, 743
0.00, 0.90, 0.40, 0.30, 741
0.00, 0.90, 0.40, 0.40, 741
0.00, 0.90, 0.40, 0.50, 741
0.00, 0.90, 0.40, 0.

0.00, 1.10, 0.40, 1.20, 741
0.00, 1.10, 0.40, 1.30, 741
0.00, 1.10, 0.40, 1.40, 741
0.00, 1.10, 0.40, 1.50, 741
0.00, 1.10, 0.50, 0.20, 744
0.00, 1.10, 0.50, 0.30, 743
0.00, 1.10, 0.50, 0.40, 743
0.00, 1.10, 0.50, 0.50, 743
0.00, 1.10, 0.50, 0.60, 743
0.00, 1.10, 0.50, 0.70, 743
0.00, 1.10, 0.50, 0.80, 743
0.00, 1.10, 0.50, 0.90, 743
0.00, 1.10, 0.50, 1.00, 743
0.00, 1.10, 0.50, 1.10, 743
0.00, 1.10, 0.50, 1.20, 743
0.00, 1.10, 0.50, 1.30, 743
0.00, 1.10, 0.50, 1.40, 743
0.00, 1.10, 0.50, 1.50, 743
0.00, 1.10, 0.60, 0.20, 743
0.00, 1.10, 0.60, 0.30, 745
0.00, 1.10, 0.60, 0.40, 745
0.00, 1.10, 0.60, 0.50, 745
0.00, 1.10, 0.60, 0.60, 745
0.00, 1.10, 0.60, 0.70, 745
0.00, 1.10, 0.60, 0.80, 745
0.00, 1.10, 0.60, 0.90, 745
0.00, 1.10, 0.60, 1.00, 745
0.00, 1.10, 0.60, 1.10, 745
0.00, 1.10, 0.60, 1.20, 745
0.00, 1.10, 0.60, 1.30, 745
0.00, 1.10, 0.60, 1.40, 745
0.00, 1.10, 0.60, 1.50, 745
0.00, 1.10, 0.70, 0.20, 745
0.00, 1.10, 0.70, 0.30, 746
0.00, 1.10, 0.70, 0.40, 746
0.00, 1.10, 0.70, 0.

...

0.20, 0.80, 0.90, 1.50, 748
0.20, 0.80, 1.00, 0.20, 750
0.20, 0.80, 1.00, 0.30, 750
0.20, 0.80, 1.00, 0.40, 750
0.20, 0.80, 1.00, 0.50, 751
0.20, 0.80, 1.00, 0.60, 751
0.20, 0.80, 1.00, 0.70, 751
0.20, 0.80, 1.00, 0.80, 752
0.20, 0.80, 1.00, 0.90, 752
0.20, 0.80, 1.00, 1.00, 752
0.20, 0.80, 1.00, 1.10, 753
0.20, 0.80, 1.00, 1.20, 753
0.20, 0.80, 1.00, 1.30, 753
0.20, 0.80, 1.00, 1.40, 753
0.20, 0.80, 1.00, 1.50, 753
0.20, 0.90, 0.20, 0.20, 744
0.20, 0.90, 0.20, 0.30, 741
0.20, 0.90, 0.20, 0.40, 741
0.20, 0.90, 0.20, 0.50, 741
0.20, 0.90, 0.20, 0.60, 741
0.20, 0.90, 0.20, 0.70, 741
0.20, 0.90, 0.20, 0.80, 741
0.20, 0.90, 0.20, 0.90, 741
0.20, 0.90, 0.20, 1.00, 741
0.20, 0.90, 0.20, 1.10, 741
0.20, 0.90, 0.20, 1.20, 741
0.20, 0.90, 0.20, 1.30, 740
0.20, 0.90, 0.20, 1.40, 740
0.20, 0.90, 0.20, 1.50, 740
0.20, 0.90, 0.30, 0.20, 743
0.20, 0.90, 0.30, 0.30, 741
0.20, 0.90, 0.30, 0.40, 741
0.20, 0.90, 0.30, 0.50, 741
0.20, 0.90, 0.30, 0.60, 741
0.20, 0.90, 0.30, 0.70, 741
0.20, 0.90, 0.30, 0.

0.20, 1.10, 0.30, 1.40, 740
0.20, 1.10, 0.30, 1.50, 740
0.20, 1.10, 0.40, 0.20, 743
0.20, 1.10, 0.40, 0.30, 741
0.20, 1.10, 0.40, 0.40, 741
0.20, 1.10, 0.40, 0.50, 741
0.20, 1.10, 0.40, 0.60, 741
0.20, 1.10, 0.40, 0.70, 741
0.20, 1.10, 0.40, 0.80, 741
0.20, 1.10, 0.40, 0.90, 741
0.20, 1.10, 0.40, 1.00, 741
0.20, 1.10, 0.40, 1.10, 741
0.20, 1.10, 0.40, 1.20, 740
0.20, 1.10, 0.40, 1.30, 740
0.20, 1.10, 0.40, 1.40, 740
0.20, 1.10, 0.40, 1.50, 740
0.20, 1.10, 0.50, 0.20, 744
0.20, 1.10, 0.50, 0.30, 742
0.20, 1.10, 0.50, 0.40, 743
0.20, 1.10, 0.50, 0.50, 743
0.20, 1.10, 0.50, 0.60, 743
0.20, 1.10, 0.50, 0.70, 743
0.20, 1.10, 0.50, 0.80, 742
0.20, 1.10, 0.50, 0.90, 742
0.20, 1.10, 0.50, 1.00, 742
0.20, 1.10, 0.50, 1.10, 742
0.20, 1.10, 0.50, 1.20, 742
0.20, 1.10, 0.50, 1.30, 742
0.20, 1.10, 0.50, 1.40, 742
0.20, 1.10, 0.50, 1.50, 742
0.20, 1.10, 0.60, 0.20, 742
0.20, 1.10, 0.60, 0.30, 745
0.20, 1.10, 0.60, 0.40, 745
0.20, 1.10, 0.60, 0.50, 745
0.20, 1.10, 0.60, 0.60, 744
0.20, 1.10, 0.60, 0.

0.20, 1.30, 0.60, 1.30, 744
0.20, 1.30, 0.60, 1.40, 744
0.20, 1.30, 0.60, 1.50, 744
0.20, 1.30, 0.70, 0.20, 746
0.20, 1.30, 0.70, 0.30, 746
0.20, 1.30, 0.70, 0.40, 746
0.20, 1.30, 0.70, 0.50, 745
0.20, 1.30, 0.70, 0.60, 745
0.20, 1.30, 0.70, 0.70, 745
0.20, 1.30, 0.70, 0.80, 745
0.20, 1.30, 0.70, 0.90, 746
0.20, 1.30, 0.70, 1.00, 746
0.20, 1.30, 0.70, 1.10, 747
0.20, 1.30, 0.70, 1.20, 747
0.20, 1.30, 0.70, 1.30, 747
0.20, 1.30, 0.70, 1.40, 747
0.20, 1.30, 0.70, 1.50, 747
0.20, 1.30, 0.80, 0.20, 748
0.20, 1.30, 0.80, 0.30, 745
0.20, 1.30, 0.80, 0.40, 746
0.20, 1.30, 0.80, 0.50, 748
0.20, 1.30, 0.80, 0.60, 748
0.20, 1.30, 0.80, 0.70, 748
0.20, 1.30, 0.80, 0.80, 748
0.20, 1.30, 0.80, 0.90, 748
0.20, 1.30, 0.80, 1.00, 748
0.20, 1.30, 0.80, 1.10, 748
0.20, 1.30, 0.80, 1.20, 748
0.20, 1.30, 0.80, 1.30, 748
0.20, 1.30, 0.80, 1.40, 748
0.20, 1.30, 0.80, 1.50, 748
0.20, 1.30, 0.90, 0.20, 747
0.20, 1.30, 0.90, 0.30, 747
0.20, 1.30, 0.90, 0.40, 748
0.20, 1.30, 0.90, 0.50, 748
0.20, 1.30, 0.90, 0.

0.20, 1.50, 0.90, 1.20, 748
0.20, 1.50, 0.90, 1.30, 748
0.20, 1.50, 0.90, 1.40, 748
0.20, 1.50, 0.90, 1.50, 748
0.20, 1.50, 1.00, 0.20, 750
0.20, 1.50, 1.00, 0.30, 748
0.20, 1.50, 1.00, 0.40, 749
0.20, 1.50, 1.00, 0.50, 749
0.20, 1.50, 1.00, 0.60, 750
0.20, 1.50, 1.00, 0.70, 750
0.20, 1.50, 1.00, 0.80, 751
0.20, 1.50, 1.00, 0.90, 752
0.20, 1.50, 1.00, 1.00, 752
0.20, 1.50, 1.00, 1.10, 752
0.20, 1.50, 1.00, 1.20, 753
0.20, 1.50, 1.00, 1.30, 753
0.20, 1.50, 1.00, 1.40, 753
0.20, 1.50, 1.00, 1.50, 753
0.30, 0.20, 0.20, 0.20, 744
0.30, 0.20, 0.20, 0.30, 740
0.30, 0.20, 0.20, 0.40, 740
0.30, 0.20, 0.20, 0.50, 740
0.30, 0.20, 0.20, 0.60, 740
0.30, 0.20, 0.20, 0.70, 740
0.30, 0.20, 0.20, 0.80, 740
0.30, 0.20, 0.20, 0.90, 740
0.30, 0.20, 0.20, 1.00, 740
0.30, 0.20, 0.20, 1.10, 740
0.30, 0.20, 0.20, 1.20, 740
0.30, 0.20, 0.20, 1.30, 740
0.30, 0.20, 0.20, 1.40, 740
0.30, 0.20, 0.20, 1.50, 740
0.30, 0.20, 0.30, 0.20, 742
0.30, 0.20, 0.30, 0.30, 740
0.30, 0.20, 0.30, 0.40, 740
0.30, 0.20, 0.30, 0.

0.30, 0.40, 0.30, 1.10, 740
0.30, 0.40, 0.30, 1.20, 740
0.30, 0.40, 0.30, 1.30, 740
0.30, 0.40, 0.30, 1.40, 740
0.30, 0.40, 0.30, 1.50, 740
0.30, 0.40, 0.40, 0.20, 742
0.30, 0.40, 0.40, 0.30, 740
0.30, 0.40, 0.40, 0.40, 740
0.30, 0.40, 0.40, 0.50, 740
0.30, 0.40, 0.40, 0.60, 740
0.30, 0.40, 0.40, 0.70, 740
0.30, 0.40, 0.40, 0.80, 740
0.30, 0.40, 0.40, 0.90, 740
0.30, 0.40, 0.40, 1.00, 740
0.30, 0.40, 0.40, 1.10, 740
0.30, 0.40, 0.40, 1.20, 740
0.30, 0.40, 0.40, 1.30, 740
0.30, 0.40, 0.40, 1.40, 740
0.30, 0.40, 0.40, 1.50, 740
0.30, 0.40, 0.50, 0.20, 743
0.30, 0.40, 0.50, 0.30, 742
0.30, 0.40, 0.50, 0.40, 742
0.30, 0.40, 0.50, 0.50, 742
0.30, 0.40, 0.50, 0.60, 742
0.30, 0.40, 0.50, 0.70, 742
0.30, 0.40, 0.50, 0.80, 742
0.30, 0.40, 0.50, 0.90, 742
0.30, 0.40, 0.50, 1.00, 742
0.30, 0.40, 0.50, 1.10, 742
0.30, 0.40, 0.50, 1.20, 742
0.30, 0.40, 0.50, 1.30, 742
0.30, 0.40, 0.50, 1.40, 742
0.30, 0.40, 0.50, 1.50, 742
0.30, 0.40, 0.60, 0.20, 743
0.30, 0.40, 0.60, 0.30, 744
0.30, 0.40, 0.60, 0.

In [32]:
sorted(scores, key = lambda x : -x[0])[:20]

[(754, 1.0, -0.04999999999999993),
 (752, 1.0, -0.09999999999999998),
 (750, 1.0, -0.1499999999999999),
 (748, 1.0, -0.19999999999999996),
 (748, 1.0, 0.0),
 (746, 1.0, 0.30000000000000004),
 (745, 1.0, 0.25),
 (745, 1.0, 0.3500000000000001),
 (745, 1.0, 0.40000000000000013),
 (745, 1.0, 0.4500000000000002),
 (744, 0.6500000000000001, 0.6500000000000001),
 (744, 0.9500000000000002, 0.5),
 (744, 1.0, -0.29999999999999993),
 (744, 1.0, -0.25),
 (744, 1.0, 0.10000000000000009),
 (744, 1.0, 0.15000000000000013),
 (744, 1.0, 0.20000000000000018),
 (744, 1.0, 0.5),
 (743, 0.8, 0.6000000000000001),
 (743, 0.8500000000000001, 0.55)]

In [189]:
sorted(scores, key = lambda x : -x[0])[:30]

[(762, 0.6000000000000001, 0.20000000000000018, 1.0, 1.0),
 (762, 0.6000000000000001, 0.6000000000000001, 1.0, 1.0),
 (762, 0.6000000000000001, 1.0, 1.0, 1.0),
 (761, 0.20000000000000018, 0.20000000000000018, 1.0, 1.0),
 (761, 0.20000000000000018, 0.6000000000000001, 1.0, 1.0),
 (761, 0.20000000000000018, 1.0, 1.0, 1.0),
 (759, 0.20000000000000018, 0.20000000000000018, 1.0, 0.6000000000000001),
 (757, 0.20000000000000018, 0.6000000000000001, 1.0, 0.6000000000000001),
 (757, 0.20000000000000018, 1.0, 1.0, 0.6000000000000001),
 (756, 1.0, 1.0, 1.0, 0.6000000000000001),
 (755, 0.6000000000000001, 0.6000000000000001, 1.0, 0.6000000000000001),
 (754, 0.20000000000000018, 0.20000000000000018, 0.6000000000000001, 1.0),
 (754,
  0.20000000000000018,
  0.6000000000000001,
  0.6000000000000001,
  0.6000000000000001),
 (754, 0.20000000000000018, 0.6000000000000001, 0.6000000000000001, 1.0),
 (754, 0.20000000000000018, 1.0, 0.6000000000000001, 0.6000000000000001),
 (754, 0.20000000000000018, 1.0, 

In [152]:
matrix_item_features = item_features[item_features.columns[1:]].values
matrix_user_features = user_features[user_features.columns[1:]].values

In [153]:
UF = matrix_user_features
IF = matrix_item_features
UI = UF.dot(IF.T)

In [169]:
get_predictions([UI[0]])

array([[284, 286,  76,  35, 287,  37,  22,  60,  65,  80,  72,  66, 200,
         58,   7,  40,  11, 148,  36,  21]])

In [138]:
np.max(UI)

0.021655600303483808